In [25]:
import pandas
import gdal
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from dbfread import DBF
import numpy as np

In [ ]:
def open_dbf_as_frame(dbf_path, encoding = "UTF-8"):
    """Opens a DBF as a pandas dataframe"""
    dbf = DBF.(dbf_path, encoding)
    return pandas.DataFrame(iter(changes_dbf))

def find_changes(change_directory):
    """Returns a list of dbf files """

In [4]:
changes_path = r"D:\conafor\changes\changes.dbf"
changes_dbf = DBF(changes_path, encoding="UTF-8")
changes_frame = pandas.DataFrame(iter(changes_dbf))

In [6]:
false_frame.columns

Index(['OBJECTID', 'FID_Raster', 'Id', 'gridcode', 'FID_Jal_LC', 'ipcc_dyna',
       'mdmx_t1_l', 'tcd_t1', 'Val_T1', 'Val_T2', 'Val_cambio', 'DNDVIT2_T1',
       'Join_Count', 'TARGET_FID', 'OBJECTID_1', 'FID_LCC_IR', 'oid_1',
       'change', 'mdmx_t1', 'mdmx_t2', 'mdmx_t1t2', 'mdmx_cls', 'prob_cmb',
       'maf_4', 'maf_5', 'maf_6', 'maf_1', 'maf_2', 'maf_3', 'tcd_t2',
       'met_t1_1', 'met_t1_3', 'met_t1_2', 'met_t1_5', 'met_t1_4', 'met_t1_7',
       'met_t1_6', 'met_t2_6', 'met_t2_7', 'met_t2_4', 'met_t2_5', 'met_t2_2',
       'met_t2_3', 'met_t2_1', 'area', 'id_1', 'Shape_Leng', 'MASC',
       'interprete', 'metodo', 'Join_Cou_1', 'TARGET_F_1', 'LC_16',
       'OBJECTID_2', 'OBJECTID_3', 'USVEGT1', 'USVEGT2', 'Shape_Le_1',
       'Shape_Area', 'Val_camb_3', 'REVISION', 'R_VAL_T1', 'R_VAL_T2',
       'R_val_camb'],
      dtype='object')

In [7]:
feature_columns = ["maf_4", "maf_5", "maf_6", "maf_1", "maf_2", "maf_3"]


In [47]:
change_features = changes_frame.filter(items=feature_columns).sample(2000).values
false_features = false_frame.filter(items=feature_columns).sample(2000).values
features = np.concatenate([change_features, false_features])

In [48]:
classes = np.concatenate(
    [np.ones(len(change_features)), np.zeros(len(false_features))])

In [49]:
print(classes.shape)
print(features.shape)

(4000,)
(4000, 6)


In [50]:
train_features, test_features, train_classes, test_classes = train_test_split(features, classes)

In [51]:
train_classes

array([ 0.,  0.,  0., ...,  0.,  0.,  1.])

In [52]:
model = RandomForestClassifier(oob_score = True, n_estimators = 100)
model.fit(train_features, train_classes)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [53]:
out_classes = model.predict(test_features)
out_classes == test_classes

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [55]:
model.feature_importances_

array([ 0.15921711,  0.21930417,  0.33974435,  0.13922548,  0.06691822,
        0.07559067])